In [1]:
import tensorflow as tf
import numpy as np
import sys
import h5py
import random
import math
from math import sqrt
import argparse

#thanks to https://gist.githubusercontent.com/kukuruza/03731dc494603ceab0c5/raw/3d708320145df0a962cfadb95b3f716b623994e0/gist_cifar10_train.py
def put_kernels_on_grid (kernel, grid_Y, grid_X, pad = 1):

    '''Visualize conv. features as an image (mostly for the 1st layer).
    Place kernel into a grid, with some paddings between adjacent filters.

    Args:
      kernel:            tensor of shape [Y, X, NumChannels, NumKernels]
      (grid_Y, grid_X):  shape of the grid. Require: NumKernels == grid_Y * grid_X
                           User is responsible of how to break into two multiples.
      pad:               number of black pixels around each filter (between them)

    Return:
      Tensor of shape [(Y+2*pad)*grid_Y, (X+2*pad)*grid_X, NumChannels, 1].
    '''

    x_min = tf.reduce_min(kernel)
    x_max = tf.reduce_max(kernel)

    kernel1 = (kernel - x_min) / (x_max - x_min)

    # pad X and Y
    x1 = tf.pad(kernel1, tf.constant( [[pad,pad],[pad, pad],[0,0],[0,0]] ), mode = 'CONSTANT')

    # X and Y dimensions, w.r.t. padding
    Y = kernel1.get_shape()[0] + 2 * pad
    X = kernel1.get_shape()[1] + 2 * pad

    channels = kernel1.get_shape()[2]

    # put NumKernels to the 1st dimension
    x2 = tf.transpose(x1, (3, 0, 1, 2))
    # organize grid on Y axis
    x3 = tf.reshape(x2, tf.stack([grid_X, Y * grid_Y, X, channels])) #3

    # switch X and Y axes
    x4 = tf.transpose(x3, (0, 2, 1, 3))
    # organize grid on X axis
    x5 = tf.reshape(x4, tf.stack([1, X * grid_X, Y * grid_Y, channels])) #3

    # back to normal order (not combining with the next step for clarity)
    x6 = tf.transpose(x5, (2, 1, 3, 0))

    # to tf.image_summary order [batch_size, height, width, channels],
    #   where in this case batch_size == 1
    x7 = tf.transpose(x6, (3, 0, 1, 2))

    # scale to [0, 255] and convert to uint8
    return tf.image.convert_image_dtype(x7, dtype = tf.uint8)

def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        meansum = tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
              stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        stdv = tf.summary.scalar('stddev', stddev)
        maxsum = tf.summary.scalar('max', tf.reduce_max(var))
        minsum = tf.summary.scalar('min', tf.reduce_min(var))
        hist = tf.summary.histogram('histogram', var)
        
        return tf.summary.merge([meansum,stdv,maxsum,minsum,hist])


def main(Learning_Rate,multiplier):

    trainfile = "/om/user/ribeirop/audiosetDL/balanced_stripped/balanced_train_segments_Coch.hdf5"
    train_averagefile = "/om/user/ribeirop/audiosetDL/balanced_stripped/balanced_train_segments_average.npy"
    testfile = trainfile# "/om/user/ribeirop/audiosetDL/eval_stripped/eval_segments_Coch.hdf5"
    test_averagefile = train_averagefile# "/om/user/ribeirop/audiosetDL/eval_stripped/eval_segments_average.npy"
    trainset = h5py.File(trainfile)
    testset = h5py.File(testfile)
    
    train_ave_coch = np.load(train_averagefile)
    test_ave_coch = np.load(test_averagefile)
    
    trainlabels_file = "/home/ribeirop/OMFOLDER/audiosetDL/MODIFIED_LABELS/balanced_train_segments_nomusic_labels_only.hdf5"
    testlabels_file = trainlabels_file#"/home/ribeirop/OMFOLDER/audiosetDL/MODIFIED_LABELS/eval_segments_nomusic_labels_only.hdf5"
    
    trainlabelsSet = h5py.File(trainlabels_file)
    testlabelsSet = h5py.File(testlabels_file)
    
    Num_Epochs = 100
    
    COCHLEAGRAM_LENGTH = int(342000/1) # full is 342000
    conv1_strides = 3
    conv2_strides = 2
    conv3_strides = 1
    conv4_strides = 1
    conv5_strides = 1
    
    limit = 128*multiplier
    
    trainsize = limit#trainset.attrs.get("size")
    testsize = limit#testset.attrs.get("size")
    
    b = 0
    batchsize = 64
    
    final_filter = 512
    
    TensorBoard_Folder = './TB/NOMUSICtrain_n{0}_lr{1}'.format(limit,Learning_Rate)
    
    full_length = final_filter * math.ceil(COCHLEAGRAM_LENGTH/171/conv1_strides/conv2_strides/conv3_strides/conv4_strides/conv5_strides/4/2)* math.ceil(171/conv1_strides/conv2_strides/conv3_strides/conv4_strides/conv5_strides/4/2)
    
    nets = {}

    with tf.Graph().as_default():
        with tf.name_scope('input'):
            nets['input_to_net'] = tf.placeholder(tf.float32, [None,COCHLEAGRAM_LENGTH], name='input_to_net')
            nets['actual_labels'] = tf.placeholder(tf.float32, [None, 526], name='actual_labels')
            nets['label_indeces'] = tf.placeholder(tf.int32, [None, 15], name='label_indeces')
            
            
            nets['accuracy'] = tf.placeholder(tf.float32, (), name='acc')
            nets['accsum'] = tf.summary.scalar("accuracy",nets['accuracy'])
            
            
            nets['reshapedCoch'] = tf.reshape(nets['input_to_net'],[-1,171,int(COCHLEAGRAM_LENGTH/171),1], name='reshape_input')

        #sess.run(tf.global_variables_initializer())
        #thing = sess.run(reshapedCoch,feed_dict={input_to_net:trainset["/coch"][0:2],actual_labels:trainset["/labels"][0:2]})



        with tf.variable_scope('conv1') as scope:
            tf.get_variable_scope().reuse_variables()
            conv1_Weights = weight_variable([9,9,1,96])
            
            layer1 = variable_summaries(conv1_Weights)
            
            grid = put_kernels_on_grid(conv1_Weights,16,6)
            nets['conv1_weight_image'] = tf.summary.image('conv1/kernels', grid, max_outputs=3)
            
            nets['conv1'] = conv2d(nets['reshapedCoch'], Weights = conv1_Weights, bias = bias_variable([96]), strides=conv1_strides, name='conv1')
            nets['maxpool1'] = maxpool2x2(nets['conv1'],k=2 , name='maxpool1')


            
            #nets['conv1_image'] = tf.reshape(nets['conv1'],[])

        with tf.name_scope('conv2'):
            conv2_Weights = weight_variable([5,5,96,256])
            nets['conv2'] = conv2d(nets['maxpool1'], Weights = conv2_Weights, bias = bias_variable([256]), strides=conv2_strides, name='conv2')
            nets['maxpool2'] = maxpool2x2(nets['conv2'],k=2 , name='maxpool2')

            layer2 = variable_summaries(conv2_Weights)
            
        with tf.name_scope('conv3'):
            conv3_Weights = weight_variable([5,5,96,256])
            nets['conv3'] = conv2d(nets['maxpool2'], Weights = weight_variable([3,3,256,512]), bias = bias_variable([512]), strides=conv3_strides, name='conv3')

        with tf.name_scope('conv4'):
            nets['conv4'] = conv2d(nets['conv3'], Weights = weight_variable([3,3,512,1024]), bias = bias_variable([1024]), strides=conv4_strides, name='conv4')        

        with tf.name_scope('conv5'):
            nets['conv5'] = conv2d(nets['conv4'], Weights = weight_variable([3,3,1024,final_filter]), bias = bias_variable([final_filter]),strides=conv5_strides, name='conv5')             
            
            nets['maxpool3'] = maxpool2x2(nets['conv5'],k=2 , name='maxpool3')
            
            nets['flattened'] = tf.reshape(nets['maxpool3'], [-1, full_length])
            
        with tf.name_scope('fc_1'):
            W_fc1 = weight_variable([full_length, 1024]) # 4,959,232
            b_fc1 = bias_variable([1024])
            nets['fc_1'] = tf.nn.relu(tf.matmul(nets['flattened'], W_fc1) + b_fc1)

            keep_prob = tf.placeholder(tf.float32)
            nets['h_fc1_drop'] = tf.nn.dropout(nets['fc_1'], keep_prob)

        with tf.name_scope('fc_2'):
            W_fc2 = weight_variable([1024, 526])
            b_fc2 = bias_variable([526])
            nets['predicted_labels'] = tf.matmul(nets['h_fc1_drop'], W_fc2) + b_fc2

            
        with tf.variable_scope("eval"):
            nets['cross_entroy'] = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels =nets['actual_labels'],logits = nets['predicted_labels'] ))
            nets['train_step'] = tf.train.AdamOptimizer(Learning_Rate).minimize(nets['cross_entroy'])

            nets['cross_entropy_summary'] = tf.summary.scalar("cross_entropy_summary",nets['cross_entroy'])
            
            #num = np.sum(nets['actual_labelsi'])
            #nets['accuracy'] = tf.metrics.mean(tf.nn.in_top_k(nets['predicted_labels'],nets['actual_labelsi'],2))

            #calculating accuracys

            _,nets['indeces'] = tf.nn.top_k(nets['predicted_labels'],15)
            prediction_sums = tf.summary.histogram('predicted_labels_histogram',nets['indeces'])
            nets['numCorrect'] = tf.shape(tf.sets.set_intersection(nets['indeces'],nets['label_indeces'],False).values)[0]
        
        merged = tf.summary.merge_all()
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            train_writer = tf.summary.FileWriter(TensorBoard_Folder, sess.graph)
            
            for epoch in range(Num_Epochs):
                print("epoch: ",epoch)
                print("training")
                
                #Train a full epoch
                startindex = 0
                endindex = startindex+batchsize
                while(endindex <= trainsize):
                    print(startindex)
                    _,cross_entropySummary,l1Sums = sess.run([nets['train_step'],nets['cross_entropy_summary'],layer1],feed_dict={nets['input_to_net']:trainset["/coch"][startindex:endindex][:,0:COCHLEAGRAM_LENGTH]-train_ave_coch[0:COCHLEAGRAM_LENGTH],nets['actual_labels']:trainlabelsSet["/labels"][startindex:endindex],keep_prob:.5})
                    train_writer.add_summary(cross_entropySummary, epoch)
                    train_writer.add_summary(l1Sums, epoch)
                    
                    startindex = endindex
                    endindex = startindex+batchsize
                    
                
                print("eval")
                #Evaluate the full epoch
                startindex = 0
                endindex = startindex+batchsize
                total_count = 0
                correct = 0
                while(endindex <= testsize):
                    print(startindex)
                    trainBatchLabels = testlabelsSet["/labels"][startindex:endindex]
                    indeces_list = []
                    for row in trainBatchLabels:
                        indeces= np.where(row==1)[0]
                        total_count = total_count + len(indeces)
                        indeces_list.append(np.pad(indeces,[0,15-len(indeces)],mode = 'constant',constant_values=-1))
                        
                    
                    addme,pred_sum = sess.run([nets['numCorrect'],prediction_sums],feed_dict={nets['input_to_net']:testset["/coch"][startindex:endindex][:,0:COCHLEAGRAM_LENGTH]-test_ave_coch[0:COCHLEAGRAM_LENGTH],nets['actual_labels']:testlabelsSet["/labels"][startindex:endindex],nets['label_indeces']:indeces_list,keep_prob:1})
                    train_writer.add_summary(pred_sum, epoch + startindex/testsize)
                    correct = correct + addme
                    
                    startindex = endindex
                    endindex = startindex+batchsize
                
                
                summary = sess.run(nets['accsum'],feed_dict={nets['accuracy']:correct/total_count})
                train_writer.add_summary(summary, epoch)
                
                image_conv1 = sess.run(nets['conv1_weight_image'])
                train_writer.add_summary(image_conv1, epoch)
    

                
                startindex = random.randint(0,testsize-batchsize)
                endindex = startindex+batchsize
                potato = sess.run(nets['indeces'],feed_dict={nets['input_to_net']:testset["/coch"][startindex:endindex][:,0:COCHLEAGRAM_LENGTH]-test_ave_coch[0:COCHLEAGRAM_LENGTH],nets['actual_labels']:testlabelsSet["/labels"][startindex:endindex],nets['label_indeces']:indeces_list,keep_prob:1})
                print()
                print("Epoch {0}, accuracy {1}".format(epoch,correct/total_count))
                #print(start)
                #print(end)
                #print(total_count)
                #print(correct)
                #print("actual: ", indeces_list)
                print("predicted :", potato)
                #print()
                #f= open("output3.csv","a")
                #f.write("loop {0}, accuracy {1}\n".format(i,correct/total_count))
                #f.close()''
                
    print("done")
            
           
    
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

#builds the components
def conv2d(inputtensor, Weights, bias, strides=1,name = None):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(inputtensor, Weights, strides=[1, strides, strides, 1], padding='SAME',name=name)
    x = tf.nn.bias_add(x, bias)
    return tf.nn.relu(x)

def maxpool2x2(x, k=2, name = None):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME',name=name)
    
if __name__ == '__main__':
    debug = True #if true, then use the default csv file to read from. 
    if debug:
        learning_rate = 1e-5
        multiplier = 10
        main(learning_rate,multiplier)
    else:
        
        parser = argparse.ArgumentParser()
        parser.add_argument('-a', '--learning_rate')
        parser.add_argument('-b', '--multiplier')

        args = vars(parser.parse_args())

        learning_rate = float(args["learning_rate"])
        multiplier = int(args["multiplier"])
        
        print(learning_rate)
        print(multiplier)

        main(learning_rate,multiplier)
        

epoch:  0
training
0
64
128
192
256
320
384
448
512
576
640
704
768
832
896
960
1024
1088
1152
1216
eval
0
64
128
192
256
320
384
448
512
576
640
704
768
832
896
960
1024
1088
1152
1216

Epoch 0, accuracy 0.025717111770524232
predicted : [[155 339 334 149 384 518 152 134 237 169 324 501 522 161 352]
 [152 519 445  18 146 407  28  29 296 334 480 149  30 310 418]
 [339 149 519 500 407 146 135 125 461 480  18 296 445 134 473]
 [384 407 149  29 339 519 480 418  14 152 155 135  30 424 275]
 [519 407 310 480 149  72 334 445  14 296 518 264 146  18 152]
 [169 152 135 350 501  29  82 381  77 149 461  14  72 143 155]
 [149  14 339 518 237  77 268 135  28 461 350 112 375 480  18]
 [480 500  41 339 135  28 143 155 384 381  18 418 473 436 519]
 [ 77 480 135 407  51 375 518 436 461 299  14 464 381 161 149]
 [152 407 149 519 146 461 334 507 125  46 268  34 500  28 135]
 [407 480 384 296 146 519 275 339 152  29 135 418 436 149 326]
 [152 339 480 149 135 334 436  72 324 169  28  77 292 434 407]
 [407 

64
128
192
256
320
384
448
512
576
640
704
768
832
896
960
1024
1088
1152
1216
eval
0
64
128
192
256
320
384
448
512
576
640
704
768
832
896
960
1024
1088
1152
1216

Epoch 2, accuracy 0.04516979887899769
predicted : [[149 375  25  75  34 316 349 281 196 135 459 268 428 358  51]
 [135 146 453   7 152 396 424 296 302 278 457 308  69 109 248]
 [135 334 169  41 427  34 149 476 201 352  25 253 369 105 449]
 [135 316 152 462 169  26  81  19 373 292 461 396 442  34  90]
 [135 407  25 206 434 289 349 171 152 445 496  71 462 480 137]
 [349 149 316 152  26 476 289 195  82  25 503 427 135 370 241]
 [135  77  51   0 334 149  39  41 290 395 289 223  75 427 296]
 [445 152 478   7 137 368 438 480 424 289 135  14 399 269 149]
 [152 296  14 135  68 427 302   7 149 188 396 142 515 352 453]
 [152 478 434  34 149 457 275  72 373 157 169 248 146   9 396]
 [ 41 135 149  34 500   0 253 152   7  77 349  51 473 450 167]
 [ 34 357  59 292  70 377 152 427 381 436   7 355 233  19 171]
 [290 349   7  14 434 449 42

64
128
192
256
320
384
448
512
576
640
704
768
832
896
960
1024
1088
1152
1216
eval
0
64
128
192
256
320
384
448
512
576
640
704
768
832
896
960
1024
1088
1152
1216

Epoch 4, accuracy 0.04780745136828223
predicted : [[135 334 128 312 504  66 352 258 328 438 307 427  78 360 217]
 [381 152 438 189 135  94 501 151 298 316 228 143 125 155  29]
 [492 206 381  34 289  51 152  72 434 294 110  89 407 388  14]
 [ 36 328  33 443 357  90  17  19  81 396 137 152 338 148 289]
 [254  97 412  24 445 171 302 337  34  51 292 381 289 149 413]
 [218 254  21 350 423 152 241 357 460 302 370 250 328  25 461]
 [296 299 231 297 268 302  36 334  65 314 275 328  59 365 500]
 [166 248 469 349 144 444 484 339 485 370 200 504 446  75 393]
 [308 445 193 437  51 478 446  15 312 163 433 143 161 266 149]
 [ 75 441   2 168  16 476 149 349 395  82  39 470 410 480 243]
 [ 75 218 370  68 169 352 445 308  30  94  91 430  34 466 156]
 [349 149 384  53 453 258 152 269 254 146 261 285 446 394 438]
 [381 519 268  72 307 461  8

64
128
192
256
320
384
448
512
576
640
704
768
832
896
960
1024
1088
1152
1216
eval
0
64
128
192
256
320
384
448
512
576
640
704
768
832
896
960
1024
1088
1152
1216

Epoch 6, accuracy 0.033630069238377844
predicted : [[287 514 156 213 336 176 348 373 160   0 340  15 401 231 512]
 [295 131 195  90 199 274 308 369 419 114 206 261 163 280 501]
 [333 174 136  39  94 199 340 100  30 505  38 351  23  50 191]
 [422 136 440 145 421 489 432  20 222 229 518 398 483  10 310]
 [196 426 320 502 336 333 303 446 489 452 130 325 218 213 215]
 [130  44 135  10 201 504 284 394  68  70 484 251  79 337 465]
 [298 394 475 332 161 130 431 446  82  23 216 248  39 512 135]
 [ 69 188 146 375  17 100 480 384 465 296 520 456 478 173 457]
 [254 355 344 314  68 430 346 130  16 155  93 192 146  70 149]
 [262 166 290 440 468 353 137  46 115 282 397 160 259 476 237]
 [254 274 452 443 405 131 285  56 342 394 439 287  27 229 389]
 [ 18 478 389 112  29 495 157 177  44  19  48 105 417 393   3]
 [161 416 153 316  29 402 1

64
128
192
256
320
384
448
512
576
640
704
768
832
896
960
1024
1088
1152
1216
eval
0
64
128
192
256
320
384
448
512
576
640
704
768
832
896
960
1024
1088
1152
1216

Epoch 8, accuracy 0.07154632377184306
predicted : [[189 456 507 258 319 364  31  43 196 519 257 207 306 118 365]
 [136 518 440  20 432 222 436 310 519 229 483  28 125 487  72]
 [136 518 440  20 432 222 436 310 519 229 483  28 125 487  72]
 [136 518 440  20 432 222 436 310 519 229 483  28 125 487  72]
 [136 518 440  20 432 222 436 310 519 229 483  28 125 487  72]
 [510 209 495 410 195 339 281  64 301 280 469 153 173 159 202]
 [467 184  55 124   7 236 515  44 320 341 485 369 330 398 197]
 [288 313 248 245  73 437 443 387 374  16  23 375 487 277 159]
 [512 513 319 114 119 515 284 523 446 206 351 243  35 257 300]
 [249   3 139  58 289  86 324 488  20   8 153 370 476 201 354]
 [136 518 440  20 432 222 436 310 519 229 483  28 125 487  72]
 [518 505 293  99 245 515 108 270 103 144 219  95 343 239 163]
 [136 518 440  20 432 222 43

...

64
128
192
256
320
384
448
512
576
640
704
768
832
896
960
1024
1088
1152
1216
eval
0
64
128
192
256
320
384
448
512
576
640
704
768
832
896
960
1024
1088
1152
1216

Epoch 42, accuracy 0.01813386086383119
predicted : [[135 152 149 480 500 339 334  77 169 146  14 445 241 296 155]
 [135 152 149 480 500 339 334  77 169 146  14 445 241 296 155]
 [135 152 149 480 500 339 334  77 169 146  14 445 241 296 155]
 [135 152 149 480 500 339 334  77 169 146  14 445 241 296 155]
 [135 152 149 480 500 339 334  77 169 146  14 445 241 296 155]
 [135 152 149 480 500 339 334  77 169 146  14 445 241 296 155]
 [135 152 149 480 500 339 334  77 169 146  14 445 241 296 155]
 [135 152 149 480 500 339 334  77 169 146  14 445 241 296 155]
 [135 152 149 480 500 339 334  77 169 146  14 445 241 296 155]
 [135 152 149 480 500 339 334  77 169 146  14 445 241 296 155]
 [135 152 149 480 500 339 334  77 169 146  14 445 241 296 155]
 [135 152 149 480 500 339 334  77 169 146  14 445 241 296 155]
 [135 152 149 480 500 339 3

64
128
192
256
320
384
448
512
576
640
704
768
832
896
960
1024
1088
1152
1216
eval
0
64
128
192
256
320
384
448
512
576
640
704
768
832
896
960
1024
1088
1152
1216

Epoch 44, accuracy 0.018793273986152326
predicted : [[135 152 149 480 500 339 334  77 169 146  14 445 241 296  51]
 [135 152 149 480 500 339 334  77 169 146  14 445 241 296  51]
 [135 152 149 480 500 339 334  77 169 146  14 445 241 296  51]
 [135 152 149 480 500 339 334  77 169 146  14 445 241 296  51]
 [135 152 149 480 500 339 334  77 169 146  14 445 241 296  51]
 [135 152 149 480 500 339 334  77 169 146  14 445 241 296  51]
 [135 152 149 480 500 339 334  77 169 146  14 445 241 296  51]
 [135 152 149 480 500 339 334  77 169 146  14 445 241 296  51]
 [135 152 149 480 500 339 334  77 169 146  14 445 241 296  51]
 [135 152 149 480 500 339 334  77 169 146  14 445 241 296  51]
 [135 152 149 480 500 339 334  77 169 146  14 445 241 296  51]
 [135 152 149 480 500 339 334  77 169 146  14 445 241 296  51]
 [135 152 149 480 500 339 

64
128
192
256
320
384
448
512
576
640
704
768
832
896
960
1024
1088
1152
1216
eval
0
64
128
192
256
320
384
448
512
576
640
704
768
832
896
960
1024
1088
1152
1216

Epoch 46, accuracy 0.018793273986152326
predicted : [[135 152 149 480 500 339 334  77 169 146  14 445 241 296  51]
 [135 152 149 480 500 339 334  77 169 146  14 445 241 296  51]
 [135 152 149 480 500 339 334  77 169 146  14 445 241 296  51]
 [135 152 149 480 500 339 334  77 169 146  14 445 241 296  51]
 [135 152 149 480 500 339 334  77 169 146  14 445 241 296  51]
 [135 152 149 480 500 339 334  77 169 146  14 445 241 296  51]
 [135 152 149 480 500 339 334  77 169 146  14 445 241 296  51]
 [135 152 149 480 500 339 334  77 169 146  14 445 241 296  51]
 [135 152 149 480 500 339 334  77 169 146  14 445 241 296  51]
 [135 152 149 480 500 339 334  77 169 146  14 445 241 296  51]
 [135 152 149 480 500 339 334  77 169 146  14 445 241 296  51]
 [135 152 149 480 500 339 334  77 169 146  14 445 241 296  51]
 [135 152 149 480 500 339 

64
128
192
256
320
384
448
512
576
640
704
768


KeyboardInterrupt: 